In [30]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
from pymystem3 import Mystem
m = Mystem()

right_ans = ['позитивный', 'позитивный', 'позитивный', 'позитивный', 'позитивный', 'негативный', 'негативный', 'негативный', 'негативный', 'негативный']
results = []   
def define_tonality(review, set_positive, set_negative):
    global right_ans
    global results
    print('hello')
    if review != '':
        pos = 0
        neg = 0
        all_tokens = ''
        py_tags = []
        tokens = nltk.word_tokenize(review)
        for token in tokens:
            if token not in '.-?,!:;—»«':
                p = m.parse(token)
                POS = p[0].tag.POS
                t = (p[0].word, POS)
                py_tags.append(t)
        ne_adjs = []
        ne_adj = []
        ochen_adjs = []
        ochen_adj = []
        for ind in range(len(py_tags)):
            if ind == len(py_tags):
                pass
            elif py_tags[ind][0] == 'не' and py_tags[ind+1][1] == 'VERB':
                ne_adj.append(py_tags[ind][0])
                ne_adj.append(py_tags[ind+1][0])
                ne_adjs.append(ne_adj)
                ne_adj = []
            elif py_tags[ind][0] == 'очень' and py_tags[ind+1][1] == 'ADJF':
                ochen_adj.append(py_tags[ind][0])
                ochen_adj.append(py_tags[ind+1][0])
                ochen_adjs.append(ochen_adj)
                ochen_adj = []
        pos += len(ochen_adjs)
        neg += len(ne_adjs)
        lemmas = m.lemmatize(all_tokens)
        for lemma in lemmas:
            if lemma in set_positive:
                pos += 1
            if lemma in set_negative:
                neg += 1
        if neg > pos:
            results.append('негативный')
        else:
            results.append('позитивный')

In [32]:
def count_accuracy(set_positive, set_negative):
    print('re')
    with open('test_pos.txt', encoding = 'utf-8') as f:
        reviews = f.read()
        for review in reviews.splitlines():
            define_tonality(review, set_positive, set_negative)
            n_grams(review)
        
    with open('test_neg.txt', encoding = 'utf-8') as f:
        reviews = f.read()
        for review in reviews.splitlines():
            define_tonality(review, set_positive, set_negative)
    print(accuracy_score(results, right_ans))  

In [33]:
def create_sets(lemmas_pos, lemmas_neg):
    fil_lemmas_pos = []
    fil_lemmas_neg = []
    for lemma in lemmas_pos:
        if lemma != ' ' and lemmas_pos.count(lemma) > 2:
            fil_lemmas_pos.append(lemma)
    for lemma in lemmas_neg:
        if lemma != ' ' and lemmas_neg.count(lemma) > 2:
            fil_lemmas_neg.append(lemma)
    set_positive = set(fil_lemmas_pos)
    set_negative = set(fil_lemmas_neg)
    count_accuracy(set_positive, set_negative)

In [34]:
def predprocessing():
    with open('positive.txt', encoding = 'utf-8') as f:
        all_tokens = ''
        reviews = f.read()
        tokens = nltk.word_tokenize(reviews)
        for token in tokens:
            if token.isalpha() and token not in stop_words:
                all_tokens = all_tokens + ' ' + token
        lemmas_pos = m.lemmatize(all_tokens)
    with open('negative.txt', encoding = 'utf-8') as f:
        all_tokens = ''
        reviews = f.read()
        tokens = nltk.word_tokenize(reviews)
        for token in tokens:
            if token.isalpha() and token not in stop_words:
                all_tokens = all_tokens + ' ' + token
        lemmas_neg = m.lemmatize(all_tokens)
    create_sets(lemmas_pos, lemmas_neg)

Модель можно было бы улучшить, добавить стоп-слова (что я сделала выше), а также возможно было полезным смотреть не только на слова, но и на коллокации

In [25]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer
def n_grams(review):
    py_tags = []
    tokens = nltk.word_tokenize(review)
    for token in tokens:
        print(token)